In [5]:
from CustomPongEnv_v0_ram import Custom_Pong
from pong_state_to_observation import pong_state_to_obs

import numpy as np
import random

from ai_agent_dpefe_fns import normalise_A, normalise_B, action_dist, softmax
import pymdp
from pymdp import utils

from scipy.stats import dirichlet
from matplotlib import pyplot as plt

random.seed(10)
np.random.seed(10);

In [4]:
env = Custom_Pong()
states = env.observation_space.shape[0]
actions = env.action_space.n

# Generative model

In [44]:
# (Hidden)Factors
# Paddle (Hypothesis)
s1_size = 42
# Ball (Hypothesis)
s2_size = 42

num_states = [s1_size, s2_size]
num_factors = len(num_states)

# Rewards
reward_modes = 3

# Controls
s1_actions = ['Stay', 'Play-Up', 'Play-Down']
s2_actions = ['Do nothing']

num_controls = [len(s1_actions), len(s2_actions)]

# Observations
#Ball-x
o1_obs_size = s1_size
#Ball-y
o2_obs_size = s1_size
#Ball-vx
o3_obs_size = 2
#Ball-vy
o4_obs_size = 2

#Paddle-pos
o5_obs_size = s2_size
#Paddle-velocity
o6_obs_size = 2
#Reward (Shock, Chocolate, and Nothing)
reward_obs_size = reward_modes

num_obs = [o1_obs_size, o2_obs_size, o3_obs_size, o4_obs_size, o5_obs_size, o6_obs_size, reward_obs_size]
num_modalities = len(num_obs)

EPS_VAL = 1e-16 # Negligibleconstant

# Likelhiood Dynamics
A = utils.random_A_matrix(num_obs, num_states)*0 + EPS_VAL

# Paddle position is initialised as fully observable
for i in range(num_states[1]):
    A[4][:,:,i] = np.eye(num_states[0]) + EPS_VAL

# Transisition dynamics
# Initialised as random becuase the agent need to learn the dynamics

B = utils.random_B_matrix(num_states, num_controls)*0 + EPS_VAL

numS = 1
for i in num_states:
    numS *= i
numA = 1
for i in num_controls:
    numA *= i

A = normalise_A(A, num_states, num_modalities)
B = normalise_B(B, num_states, num_controls)

# Prior preferences for biasing the generative model to control behaviour

## State to decision mapping

# Policy state mapping C
C = utils.obj_array_zeros([1])
C[0] = np.random.rand(numA, numS)*0 + EPS_VAL
C = normalise_A(C, 1, 1)

#Prior over states
D = utils.obj_array_uniform(num_states)

A_naive = A
B_naive = B